In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    roc_auc_score, 
    brier_score_loss,
    classification_report, 
)

# Train Models on Actions

In [ ]:
actions = pd.read_csv('data/processed_actions.csv')

In [ ]:
### PREPARE DATA FOR MODELING

# Define base features
base_features = [
    'ts_minute', 'ts_end', 'play_pattern', 'possession_team', 'team', 'player', 'type',
    'location_x', 'location_y', 'end_location_x', 'end_location_y',  
    'body_part', 'outcome', 'team_score', 'opponent_score', 
    'distance_x', 'distance_y', 'goal_diff_start', 'goal_diff_end',
    'distance_goal', 'angle_goal', 
    'distance_goal_end', 'angle_goal_end', 'under_pressure'
]

# Parameters
r = 3   # number of previous actions to include
k = 10  # look ahead horizon for target columns

# Build feature columns list
feature_cols = []
for feat in base_features:
    if feat in actions.columns:
        feature_cols.append(feat)
    for i in range(1, r):
        col = f"{feat}_prev_{i}"
        if col in actions.columns:
            feature_cols.append(col)

# Define targets
score_target = f'team_scores_in_{k}'
concede_target = f'team_concedes_in_{k}'

if score_target not in actions.columns or concede_target not in actions.columns:
    raise ValueError(f"Target columns for k={k} are missing.")

# Split data so that 2021,2022 are train, 2023 is test
train_data = actions[actions['match_id'] < 3881483]
test_data = actions[actions['match_id'] >= 3881483] 

print(f"Train observations: {len(train_data)} ({len(train_data['match_id'].unique())} matches)")
print(f"Test observations: {len(test_data)} ({len(test_data['match_id'].unique())} matches)")

# Prepare features
X_train = train_data[feature_cols].copy()
X_test = test_data[feature_cols].copy()

# Prepare targets
y_score_train = train_data[score_target].astype(int)
y_score_test = test_data[score_target].astype(int)

y_concede_train = train_data[concede_target].astype(int)
y_concede_test = test_data[concede_target].astype(int)

y_match_train = (train_data['match_outcome'] == 'win').astype(int)
y_match_test = (test_data['match_outcome'] == 'win').astype(int)

y_possession_scores_train = train_data['possession_scores'].astype(int)
y_possession_scores_test = test_data['possession_scores'].astype(int)

y_possession_concedes_train = train_data['possession_concedes'].astype(int)
y_possession_concedes_test = test_data['possession_concedes'].astype(int)

# Identify categorical vs. numeric columns
categorical_cols = [col for col in feature_cols if (col == "team" or "team_prev" in col or 'possession_team' in col or
                                                   "player" in col or "type" in col or 
                                                   "body_part" in col or "outcome" in col
                                                   or 'play_pattern' in col)]
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Convert categorical columns to string
for col in categorical_cols:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
### Model Selection

# Function to run randomized search CV for a specific model
def run_randomized_search(X, y, model, param_dist, cv_strategy, n_iter=10, scoring='neg_brier_score', n_jobs=-1, verbose=3):
    """
    Run randomized search cross-validation for a specific model.
    
    Parameters:
    -----------
    X : array-like
        Feature dataset
    y : array-like
        Target variable
    model : estimator object
        The model to optimize
    param_dist : dict
        Parameter distribution for randomized search
    n_iter : int, default=100
        Number of parameter settings sampled
    cv : int, default=5
        Number of cross-validation folds
    scoring : str, default='balanced_accuracy'
        Scoring metric
    n_jobs : int, default=-1
        Number of parallel jobs (-1 means using all processors)
    
    Returns:
    --------
    RandomizedSearchCV object
    """
    # Set up randomized search
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=cv_strategy,
        scoring=scoring,
        n_jobs=n_jobs,
        random_state=42,
        verbose=verbose,
        return_train_score=True
    )
    
    # Fit the model
    random_search.fit(X, y)
    
    return random_search

# Helper function to compute 3 metrics of interest
def compute_metrics(y, y_pred_proba):
    metrics_dict = {
        'accuracy': accuracy_score(y, (y_pred_proba>=0.5).astype(int)),
        'neg_brier_score': brier_score_loss(y, y_pred_proba),
        'roc_auc': roc_auc_score(y, y_pred_proba)
    }
    print(metrics_dict)
    return metrics_dict

# Function to compare all models after randomized search
def compare_models(results_dict, X_dict, y, cv_strategy, metrics=['accuracy', 'neg_brier_score', 'roc_auc']):
    """
    Compare the best results from each model.
    """
    comparison = {}
    
    for name, result in results_dict.items():
        best_model = result.best_estimator_
        X = X_dict[name]
        
        # Cross-validate with best parameters
        y_pred_proba = np.zeros_like(y, dtype=float)
        for train_idx, test_idx in cv_strategy.split(X, y):
            # Handle both DataFrame and numpy array
            if isinstance(X, pd.DataFrame):
                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            else:
                X_train, X_test = X[train_idx], X[test_idx]
            
            y_train = y[train_idx]
            best_model.fit(X_train, y_train)
            y_pred_proba[test_idx] = best_model.predict_proba(X_test)
        
        # Calculate metrics
        metrics_dict = compute_metrics(y, y_pred_proba)
        
        comparison[name] = {
            'CV Score': result.best_score_,
            'Best Params': result.best_params_,
            **{m: metrics_dict[m] for m in metrics},
            'Std Test Score': np.std(result.cv_results_['mean_test_score'])
        }
    
    return pd.DataFrame(comparison).T

# Main function to run the entire model selection process
def run_model_selection(models, X, y, cv_strategy, scoring='neg_brier_score', n_iter=100, n_jobs=-1):
    """
    Run model selection using randomized search CV for multiple models.
    
    Parameters:
    -----------
    X : array-like
        Feature dataset
    y : array-like
        Target variable
    n_iter : int, default=100
        Number of parameter settings sampled for each model
    class_weight : dict or 'balanced', default=None
        Class weights for models that support it
    
    Returns:
    --------
    dict
        Dictionary containing model results
    pandas.DataFrame
        Model comparison DataFrame
    """

    # Check class imbalance
    unique, counts = np.unique(y, return_counts=True)
    imbalance_ratio = max(counts) / min(counts)
    print(f"Class distribution: {dict(zip(unique, counts))}")
    print(f"Imbalance ratio: {imbalance_ratio:.2f}")
    
    # Run randomized search for each model
    results = {}
    X_dict = {}
    
    for name, config in models.items():
        print(f"\nRunning randomized search for {name}...")
        results[name] = run_randomized_search(
            config['X'], y, 
            config['model'], 
            config['params'], 
            n_iter=n_iter, 
            n_jobs=n_jobs,
            scoring=scoring,
            cv_strategy=cv_strategy
        )
        X_dict[name] = config['X']
    
    # Compare models
    comparison_df = compare_models(results, X_dict, y, cv_strategy=cv_strategy)
    
    # Print best model
    best_model_name = comparison_df[scoring].idxmax()
    print(f"\nBest model: {best_model_name} with {scoring}: {comparison_df.loc[best_model_name, scoring]:.4f}")
    
    return results, comparison_df, X_dict

def eval_best_model(results, comparison, scoring, X_dict, y, cv_strategy):
    # Display comparison
    print("\nModel Comparison:")
    print(comparison)

    # Visualize results
    plt.figure(figsize=(12, 6))
    comparison[scoring].sort_values().plot(kind='barh')
    plt.title(f'Model Comparison - {scoring}')
    plt.xlabel(scoring)
    plt.tight_layout()
    plt.show()

    # Get the best overall model
    best_model_name = comparison[scoring].idxmax()
    best_model = results[best_model_name]

    # Detailed evaluation of the best model
    print(f"\nDetailed evaluation of the best model ({best_model_name}):")
    best_estimator = best_model.best_estimator_

    print('Best model parameters:')
    print(comparison['Best Params'].iloc[0])

    y_pred = cross_val_predict(best_estimator, 
                                X_dict[best_model_name], 
                                y, 
                                cv=cv_strategy)

    print("\nClassification Report:")
    print(classification_report(y, y_pred))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()

In [ ]:
# Set up 5-fold CV
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
### Run CV metrics on the 5 models
metrics = {
    'Action Score Metrics:': y_score_train,
    'Action Concede Metrics:': y_concede_train,
    'Possession Score Metrics:': y_possession_scores_train,
    'Possession Concede Metrics:': y_possession_concedes_train,
    'Match Outcome Metrics:': y_match_train,
}

# Loop over each metric, train the classifier, compute CV predictions, and report metrics.
for label, y in metrics.items():
    # Same settings for each
    cb = CatBoostClassifier(
        random_state=42,
        verbose=1,
        cat_features=categorical_cols,
        iterations=200,
        depth=8,
        l2_leaf_reg=3,
        random_strength=1.56
    )
    y_pred_proba = cross_val_predict(
        estimator=cb,
        X=X_train,
        y=y,
        cv=cv_strategy,
        method='predict_proba'
    )[:, 1]
    
    print(label)
    compute_metrics(y, y_pred_proba)

In [ ]:
### Training and evaluation setup
def simple_eval(X, y, model):
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]  # Probability of positive class
    
    # Calculate metrics
    acc = accuracy_score(y, y_pred)
    auc = roc_auc_score(y, y_pred_proba)
    brier = brier_score_loss(y, y_pred_proba)
    
    # Print metrics
    print(f"Accuracy: {acc}")
    print(f"AUC Score: {auc}")
    print(f"Brier Score: {brier}")
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    return acc, auc, brier

def train_and_evaluate(X_train, y_train, X_test, y_test, model_class=CatBoostClassifier,
                       model_params=None, model_name="Model", feature_names=None):
    """Generic function to train and evaluate a model with AUC and Brier score"""
    
    # Training
    model = model_class(**model_params)
    model.fit(X_train, y_train)

    # Evaluation
    print(f'Train eval for {model_name}')
    simple_eval(X_train, y_train, model)
    print(f'Test eval for {model_name}')
    test_acc, test_auc, test_brier = simple_eval(X_test, y_test, model)
    
    # Plot feature importance if feature names are provided
    if feature_names is not None:
        # Get CatBoost feature importances
        feature_importances = model.get_feature_importance()
        plot_feature_importances(feature_importances, feature_names, f'Feature Importances: {model_name}')
    
    return model, test_acc, test_auc, test_brier

def plot_feature_importances(importances, feature_names, title, top_n=20):
    # Convert feature_names to list if it's not already
    feature_names = list(feature_names)
    # Create a DataFrame for easier sorting
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    })
    
    # Sort by importance and get top n features
    importance_df = importance_df.sort_values('importance', ascending=False)
    top_n = min(top_n, len(importance_df))
    importance_df = importance_df.head(top_n).iloc[::-1]
    
    # Create a horizontal bar plot
    plt.figure(figsize=(10, 6))
    plt.barh(np.arange(top_n), importance_df['importance'], align='center')
    plt.yticks(np.arange(top_n), importance_df['feature'])
    plt.xlabel('Feature Importance')
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [ ]:
### Run the training and evaluation

# Define common feature names and parameters for all models
feature_names = X_train.columns.tolist()
catboost_params = {
    'iterations': 200,
    'depth': 8,
    'random_state': 42,
    'l2_leaf_reg': 3, 
    'random_strength':1.56,
    'cat_features':categorical_cols
}

# Model 1: Team Scores
catboost_score, acc_score, auc_score, brier_score = train_and_evaluate(
    X_train, y_score_train,
    X_test, y_score_test,
    model_params=catboost_params,
    model_name=f"Team Scores in {k}",
    feature_names=feature_names 
)
# Model 2: Team Concedes
catboost_concede, acc_concede, auc_concede, brier_concede = train_and_evaluate(
    X_train, y_concede_train,
    X_test, y_concede_test,
    model_params=catboost_params,
    model_name=f"Team Concedes in {k}",
    feature_names=feature_names 
)
# Model 3: Match Outcome
catboost_match, acc_match, auc_match, brier_match = train_and_evaluate(
    X_train, y_match_train,
    X_test, y_match_test,
    model_params=catboost_params,
    model_name="Match Outcome",
    feature_names=feature_names 
)
# Model 4: Possession Scores
catboost_poss_score, acc_poss_score, auc_poss_score, brier_poss_score = train_and_evaluate(
    X_train, y_possession_scores_train,
    X_test, y_possession_scores_test,
    model_params=catboost_params,
    model_name="Possession Scores",
    feature_names=feature_names 
)
# Model 5: Possession Concedes
catboost_poss_concede, acc_poss_concede, auc_poss_concede, brier_poss_concede = train_and_evaluate(
    X_train, y_possession_concedes_train,
    X_test, y_possession_concedes_test,
    model_params=catboost_params,
    model_name="Possession Concedes",
    feature_names=feature_names 
)

# Collect results in a DataFrame for easy comparison
results = pd.DataFrame({
    'Model': [f"Team Scores in {k}", f"Team Concedes in {k}", "Match Outcome", 
              "Possession Scores", "Possession Concedes"],
    'Accuracy': [acc_score, acc_concede, acc_match, acc_poss_score, acc_poss_concede],
    'AUC Score': [auc_score, auc_concede, auc_match, auc_poss_score, auc_poss_concede],
    'Brier Score': [brier_score, brier_concede, brier_match, brier_poss_score, brier_poss_concede]
})

print("\nModel Performance Summary:")
print(results)

In [ ]:
def add_prediction_probabilities(data, X_processed, models, prob_column_names):
    """
    Add prediction probabilities from various models to a dataset.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        The dataset to add probabilities to
    X_processed : numpy.ndarray or scipy.sparse.matrix
        Processed features
    models : list
        List of trained classifier models with predict_proba method
    prob_column_names : list
        List of column names for the probability outputs
        
    Returns:
    --------
    pandas.DataFrame
        Original dataframe with probability columns added
    """
    # Make a copy to avoid modifying the original
    result = data.copy()
    
    # For each model and column name, add the positive class probability
    for model, col_name in zip(models, prob_column_names):
        result[col_name] = model.predict_proba(X_processed)[:, 1]
    
    return result

# Specify models
models = [catboost_score, catboost_concede, catboost_match, catboost_poss_score, catboost_poss_concede]
prob_columns = ['score_prob', 'concede_prob', 'win_prob', 'poss_score_prob', 'poss_concede_prob']

# Add probabilities to training and test data
train_data = add_prediction_probabilities(
    train_data, X_train, models, prob_columns
)
test_data = add_prediction_probabilities(
    test_data, X_test, models, prob_columns
)

In [ ]:
def compute_metrics(data):
    """
    Compute VAEP, WPA, and GPA metrics for a given dataset, accounting for team changes within each match.
    
    Parameters:
    - data (pd.DataFrame): DataFrame containing 'match_id', 'team', 'score_prob', 'concede_prob', 
                           'win_prob', 'poss_score_prob', and 'poss_concede_prob' columns.
                           Assumes data is sorted by 'match_id' and action sequence within each match.
    
    Returns:
    - pd.DataFrame: DataFrame with added 'VAEP', 'WPA', and 'GPA' columns.
    """
    # Shift probabilities and team within each match_id to get previous action data
    data['prev_score_prob'] = data.groupby('match_id')['score_prob'].shift()
    data['prev_concede_prob'] = data.groupby('match_id')['concede_prob'].shift()
    data['prev_win_prob'] = data.groupby('match_id')['win_prob'].shift()
    data['prev_poss_score_prob'] = data.groupby('match_id')['poss_score_prob'].shift()
    data['prev_poss_concede_prob'] = data.groupby('match_id')['poss_concede_prob'].shift()
    data['prev_team'] = data.groupby('match_id')['team'].shift()
    
    # Calculate team_change: 1 if the team changed from the previous action or if it's the first action in the match
    data['team_change'] = (data['team'] != data['prev_team']).astype(int)
    
    # Adjust previous probabilities based on team change
    # If team_change == 0 (same team), use the previous probabilities as is
    # If team_change == 1 (team changed or first action), swap score and concede probabilities
    data['adjusted_prev_score_prob'] = np.where(
        data['team_change'] == 0,
        data['prev_score_prob'],
        data['prev_concede_prob']
    )
    data['adjusted_prev_concede_prob'] = np.where(
        data['team_change'] == 0,
        data['prev_concede_prob'],
        data['prev_score_prob']
    )
    data['adjusted_prev_win_prob'] = np.where(
        data['team_change'] == 0,
        data['prev_win_prob'],
        1 - data['prev_win_prob']  # For win probability, invert it when team changes
    )
    data['adjusted_prev_poss_score_prob'] = np.where(
        data['team_change'] == 0,
        data['prev_poss_score_prob'],
        data['prev_poss_concede_prob']
    )
    data['adjusted_prev_poss_concede_prob'] = np.where(
        data['team_change'] == 0,
        data['prev_poss_concede_prob'],
        data['prev_poss_score_prob']
    )
    
    # Compute the metrics as differences from adjusted previous probabilities
    data['VAEP'] = (
        (data['score_prob'] - data['adjusted_prev_score_prob']) -
        (data['concede_prob'] - data['adjusted_prev_concede_prob'])
    )
    data['GPA'] = (
        (data['poss_score_prob'] - data['adjusted_prev_poss_score_prob']) -
        (data['poss_concede_prob'] - data['adjusted_prev_poss_concede_prob'])
    )
    data['WPA'] = data['win_prob'] - data['adjusted_prev_win_prob']
    
    # For the first action of each match (where previous values are NaN), use current probabilities
    data['VAEP'] = data['VAEP'].fillna(data['score_prob'] - data['concede_prob'])
    data['GPA'] = data['GPA'].fillna(data['poss_score_prob'] - data['poss_concede_prob'])
    data['WPA'] = data['WPA'].fillna(data['win_prob'])
    
    return data

# Compute metrics for training and test data
train_metrics = compute_metrics(train_data)
test_metrics = compute_metrics(test_data)

In [ ]:
### Experiments with adding GPA as a feature

# Model 6: WPA1
# Set up reduced set of features
modified_params = catboost_params.copy()
modified_params.pop('cat_features')
print(modified_params)
# Train and evaluate model
catboost_wpa1, acc_wpa1, auc_wpa1, brier_wpa1 = train_and_evaluate(
    train_data[['GPA']], y_match_train,
    test_data[['GPA']], y_match_test,
    model_params=modified_params,
    model_name="WPA1",
)

# Model 7: WPA2
# Prepare features
X_train_gpa = X_train.copy()
X_test_gpa = X_test.copy()
X_train_gpa['GPA'] = train_metrics['GPA']
X_test_gpa['GPA'] = test_metrics['GPA']
numeric_cols = numeric_cols + ['GPA']
# Get new feature names
feature_names_gpa = X_train_gpa.columns
# Train and evaluate model
catboost_wpa2, acc_wpa2, auc_wpa2, brier_wpa2 = train_and_evaluate(
    X_train_gpa, y_match_train,
    X_test_gpa, y_match_test,
    model_params=catboost_params,
    model_name="WPA2",
    feature_names=feature_names_gpa
)

# Use the above models to compute WPA1 and WPA2
metric_dict = {'WPA1': [catboost_wpa1, train_data[['GPA']], test_data[['GPA']]], 'WPA2': [catboost_wpa2,X_train_gpa,X_test_gpa]}
def compute_wpa_new(data, model, metric, features):
    """
    Compute alternative WPA metrics
    """
    # Compute raw probabilities for the current action
    data[f'{metric}_raw'] = model.predict_proba(features)[:, 1]
    # Shift raw probabilities to get the previous action's value within each match
    data[f'prev_{metric}_raw'] = data.groupby('match_id')[f'{metric}_raw'].shift()
    data[f'adjusted_prev_{metric}'] = np.where(
        data['team_change'] == 0,
        data[f'prev_{metric}_raw'],
        1 - data[f'prev_{metric}_raw']
    )
    # Compute the metric as change in win probability, filling NaN with raw value
    metric_values = (data[f'{metric}_raw'] - data[f'adjusted_prev_{metric}']).fillna(data[f'{metric}_raw'])
    
    return metric_values

# Compute on training and test sets
for metric, (model, train_features, test_features) in metric_dict.items():
    train_metrics[metric] = compute_wpa_new(train_data, model, metric, train_features)
    test_metrics[metric] = compute_wpa_new(test_data, model, metric, test_features)

In [ ]:
# Drop unnecessary columns for both datasets
drop_cols = [col for col in train_metrics.columns if "prev" in col or "scores_in" in col or "concedes_in" in col]
train_metrics = train_metrics.drop(columns=drop_cols)
test_metrics = test_metrics.drop(columns=drop_cols)

In [ ]:
# # Save metrics
# train_metrics.to_csv('data/train_metrics.csv', index=False)
# test_metrics.to_csv('data/test_metrics.csv', index=False)

In [ ]:
#### CROSS VALIDATION CODE
# # Example for match outcome model
# X = X_train 
# y = y_match_train
# # Define models and their parameter distributions
# models = {
#     'CatBoost': {
#         'model': CatBoostClassifier(random_state=42, verbose=1, cat_features=categorical_cols, auto_class_weights='Balanced'),
#         'params': {
#             'iterations': randint(100, 400),
#             'depth': randint(4, 10),
#             'l2_leaf_reg': randint(2, 10),
#             'random_strength': uniform(0, 10),
#             # 'border_count': randint(32, 255),
#         },
#         'X': X  # Original features
#     },
# }
# results, comparison, X_dict = run_model_selection(models=models, scoring='neg_brier_score', X=X, y=y, n_iter=3, n_jobs=-1, cv_strategy=cv_strategy) 
# eval_best_model(results, comparison, 'neg_brier_score', X_dict, y, cv_strategy=cv_strategy)